In [2]:
import pandas as pd

In [3]:
# reading the resale flat prices data
df = pd.read_csv('resale-flat-prices/resale-flat-prices-based-on-registration-date-from-jan-2015-onwards.csv')
print(df.shape)
df.head()

(80645, 11)


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0


In [4]:
# create new column, combining 'street_name' and 'singapore' to make it more accurate for geocoding
# if time permits, combining 'block', 'street_name' and 'singapore' would be more accurate
df['address'] = df['street_name'].astype(str) + ' singapore'
print(df['address'].head())
df['address'].nunique()

0     ANG MO KIO AVE 4 singapore
1    ANG MO KIO AVE 10 singapore
2     ANG MO KIO AVE 4 singapore
3    ANG MO KIO AVE 10 singapore
4    ANG MO KIO AVE 10 singapore
Name: address, dtype: object


529

In [7]:
# getting a df of unique streets
unique_streets = df['address'].unique() # returns list
streets_df = pd.DataFrame(unique_streets, columns=['address'])

# create new columns (probably don't have to do this)
streets_df['lat'] = None
streets_df['lng'] = None
streets_df['postal'] = None

streets_df.head()

,address,lat,lng,postal
0,ANG MO KIO AVE 4 singapore,None,None,None
1,ANG MO KIO AVE 10 singapore,None,None,None
2,ANG MO KIO AVE 5 singapore,None,None,None
3,ANG MO KIO AVE 8 singapore,None,None,None
4,ANG MO KIO AVE 1 singapore,None,None,None


In [ ]:
import geocoder

# loop through each row in the streets_df and geocode it, obtaining the necessary parameters and updating streets_df
# probably a faster way to do this through .apply?

for index, row in streets_df.iterrows():
    address = streets_df.iloc[index]['address']
    # get your Bing Maps key from https://docs.microsoft.com/en-us/bingmaps/getting-started/bing-maps-dev-center-help/getting-a-bing-maps-key
    g = geocoder.bing(address, key='<insert your Bing Maps key>')
    if 'lat' in g.json:
        lat = g.json['lat']
        streets_df.at[index, 'lat'] = lat
        
    if 'lng' in g.json:
        lng = g.json['lng']
        streets_df.at[index, 'lng'] = lng
    
    if 'postal' in g.json:
        postal = g.json['postal']
        streets_df.at[index, 'postal'] = postal
    
    # just a counter to keep track of which records I am at now
    if index % 10 == 0:
        print(index)

In [ ]:
# save this streets_df to .csv
# result: streets with their lat and lng
streets_df.to_csv('address_data.csv')

In [13]:
# reading the mrt names data
mrt = pd.read_csv('mrt_names.csv')
mrt.head()

,mrt
0,Jurong East
1,Bukit Batok
2,Bukit Gombak
3,Choa Chu Kang
4,Yew Tee


In [14]:
# combines 'mrt_names' with 'mrt singapore' for more accurate geocoding
# create new columns (probably don't have to do this)
mrt['mrt_names'] = mrt['mrt'] + ' mrt singapore'
mrt['lat'] = None
mrt['lng'] = None
mrt.head()

,mrt,mrt_names,lat,lng
0,Jurong East,Jurong East mrt singapore,None,None
1,Bukit Batok,Bukit Batok mrt singapore,None,None
2,Bukit Gombak,Bukit Gombak mrt singapore,None,None
3,Choa Chu Kang,Choa Chu Kang mrt singapore,None,None
4,Yew Tee,Yew Tee mrt singapore,None,None


In [ ]:
import requests
import json

# loop through each row in the mrt and geocode it, obtaining the necessary parameters and updating mrt
# probably a faster way to do this through .apply?

for index, row in mrt.iterrows():
    mrt_name = mrt.iloc[index]['mrt_names']
    # used API from OneMap Singapore (https://docs.onemap.sg/#search) rather than Bing Maps, OneMap seemed more accurate
    response = requests.get('https://developers.onemap.sg/commonapi/search?searchVal=' + mrt_name + '&returnGeom=Y&getAddrDetails=Y&pageNum=1')
    data = json.loads(response.text)
    
    if 'LATITUDE' in data['results'][0]:
        lat = data['results'][0]['LATITUDE']
        mrt.at[index, 'lat'] = lat
        
    if 'LONGTITUDE' in data['results'][0]:
        lng = data['results'][0]['LONGTITUDE']
        mrt.at[index, 'lng'] = lng
    
    # just a counter to keep track of which records I am at now
    if index % 10 == 0:
        print(index)

In [ ]:
# save this mrt to .csv
# result: mrt stations with their lat and lng
mrt.to_csv('mrt_data.csv')

In [20]:
# create new columns for streets_df
streets_df['nearest_mrt'] = None
streets_df['nearest_distance'] = None

streets_df.head()

,address,lat,lng,postal,nearest_mrt,nearest_distance
0,ANG MO KIO AVE 4 singapore,1.37741,103.84,None,None,None
1,ANG MO KIO AVE 10 singapore,1.36881,103.857,None,None,None
2,ANG MO KIO AVE 5 singapore,1.37637,103.85,None,None,None
3,ANG MO KIO AVE 8 singapore,1.371,103.849,None,None,None
4,ANG MO KIO AVE 1 singapore,1.36643,103.836,None,None,None


In [ ]:
import geopy.distance

# nested for loops, for each street, loop through list of mrt station to find the nearest mrt
# using Haversine formula and inbuilt package to calculate distance between two sets of (lat, lng)

for index, row in streets_df.iterrows():
    nearest_mrt = ''
    nearest_mrt_distance = 999999
    
    address_lat = streets_df.iloc[index]['lat']
    address_lng = streets_df.iloc[index]['lng']
    address = (address_lat, address_lng)
    
    for index2, row2 in mrt.iterrows():      
        mrt_lat = mrt.iloc[index2]['lat']
        mrt_lng = mrt.iloc[index2]['lng']
        mrt_address = (mrt_lat, mrt_lng)
        
        distance_between = geopy.distance.distance(address, mrt_address).meters
        if distance_between < nearest_mrt_distance:
            nearest_mrt_distance = distance_between
            nearest_mrt = mrt.iloc[index2]['mrt_names']
    
    streets_df.at[index, 'nearest_mrt'] = nearest_mrt
    streets_df.at[index, 'nearest_distance'] = nearest_mrt_distance
    
    # just a counter to keep track of which records I am at now
    if index % 25 == 0:
        print(index)

In [26]:
streets_df.head()

                       address      lat      lng postal  \
0   ANG MO KIO AVE 4 singapore  1.37741   103.84   None   
1  ANG MO KIO AVE 10 singapore  1.36881  103.857   None   
2   ANG MO KIO AVE 5 singapore  1.37637   103.85   None   
3   ANG MO KIO AVE 8 singapore    1.371  103.849   None   
4   ANG MO KIO AVE 1 singapore  1.36643  103.836   None   

                  nearest_mrt nearest_distance  
0  Yio Chu Kang mrt singapore          740.455  
1    Ang Mo Kio mrt singapore          837.063  
2    Ang Mo Kio mrt singapore          771.931  
3    Ang Mo Kio mrt singapore          191.985  
4    Ang Mo Kio mrt singapore          1541.43  


In [204]:
# save street_df to .csv
# result: street names with nearest mrt and distance to it
streets_df.to_csv('address_data_with_nearest_mrt.csv')